In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-12-03 17:57:06.576315


In [2]:
from transformers import AutoImageProcessor, ViTMAEForPreTraining
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split 
import glob
import cv2
from tqdm.notebook import tqdm_notebook

def resizeImage(inDir,outDir):
    target_size = (224, 224)
    print(inDir,outDir)
    curDir = os.getcwd()
    print("Current dir:",curDir)
    inDirFull = curDir + "/" + inDir + "/*" 
    print("inDIR =", inDirFull)
    outDirFull = curDir + "/" + outDir 
    print("outDIR =", outDirFull)

    image_files_normal = glob.glob(inDirFull)
    # Iterate through each image in the dataset
    for image_path_normal in image_files_normal:

        # Read the image
        image_normal = cv2.imread(image_path_normal)

        # Resize the image to the desired size (e.g., 224x224)
        resized_image_normal = cv2.resize(image_normal, target_size)

        # Normalize pixel values to be between 0 and 1
        #normalized_image = resized_image / 255.0

        # Save the preprocessed image
        filename = os.path.basename(outDirFull)
        filename2 = os.path.basename(image_path_normal)
        output_filepath_normal = os.path.join(outDirFull, filename2)
        #print(output_filepath_normal,filename2)
        cv2.imwrite(output_filepath_normal, resized_image_normal)



def load_images(source_dir, label):
    print("Load images ",source_dir, label)
    images = []
    labels = []
    
    for filename in os.listdir(source_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(source_dir, filename)
            images.append(img_path)
            labels.append(label)
    return images, labels

def process_images(image_paths):
    for path in tqdm_notebook(image_paths,  
                         desc="Processing images"):
    # any code processing the elements in the iterable
  
#    for path in image_paths:
        image = Image.open(path)
        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        
        loss = outputs.loss
        mask = outputs.mask
        ids_restore = outputs.ids_restore
 

#resizeImage("OSCC","OSCC_resize")

normal_images, normal_labels = load_images("Normal_resize", 0)
oscc_images, oscc_labels = load_images("OSCC_resize", 1)

all_images = normal_images + oscc_images
all_labels = normal_labels + oscc_labels
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-base')
model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base')


process_images(X_train)
process_images(X_val)
process_images(X_test)


Load images  Normal_resize 0
Load images  OSCC_resize 1


Processing images:   0%|          | 0/194 [00:00<?, ?it/s]

Processing images:   0%|          | 0/65 [00:00<?, ?it/s]

Processing images:   0%|          | 0/65 [00:00<?, ?it/s]

In [3]:
print("Ended: ", datetime.now())

print('\nTime elapsed both (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))



Ended:  2023-12-03 18:01:24.935616

Time elapsed both (hh:mm:ss.ms) 0:04:18.360832
